## FAISS 벡터스토어 및 Amazon Bedrock을 사용한 RAG

In [1]:
install_needed=False  # should only be True once

In [2]:
import sys
import IPython

if install_needed:
  print("installing deps and restarting kernel")
  !{sys.executable} -m pip install -U pip
  !{sys.executable} -m pip install -U sagemaker
  !{sys.executable} -m pip install "faiss-cpu>=1.7,<2"
  !{sys.executable} -m pip install langchain==0.0.279

  # IPython.Application.instance().kernel.do_shutdown(True)

In [3]:
import os, sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import print_ww

### 1. Bedrock Cient 생성

In [4]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

boto3_bedrock = bedrock.get_bedrock_client(
  assumed_role=None,
  endpoint_url=None,
  region='us-east-1'
)

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


### 2. Titan Embedding 및 LLM인 Claude-v2 모델 로딩

#### LLM Loading

In [5]:
from langchain.llms import Bedrock

llm_text = Bedrock(
  model_id="anthropic.claude-v2",
  client=boto3_bedrock,
  model_kwargs={
    'max_tokens_to_sample': 512
  }
)
llm_text

Bedrock(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<botocore.client.BedrockRuntime object at 0x7f38955f2a40>, region_name=None, credentials_profile_name=None, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 512}, endpoint_url=None)

#### Embedding Model 선택

In [6]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [7]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpoint_name=None):
  if is_bedrock_embeddings:
    # We will be using the Titan Embedding Model to generate our Embeddings
    from langchain.embeddings import BedrockEmbeddings

    llm_emb = BedrockEmbeddings(
      client=boto3_bedrock,
      model_id=bedrock_info.get_model_id(model_name="Titan-Embeddings-G1")
    )
    print("Bedrock Embeddings Model loaded")

  elif is_KoSimCSERobert:
    LLMEmbHandler = KoSimCSERobertaContentHandler()
    endpoint_name_emb = endpoint_name
    llm_emb = SagemakerEndpointEmbeddingsJumpStart(
      endpoint_name=endpoint_name,
      region_name=aws_region,
      content_handler=LLMEmbHandler
    )
    print("KoSimCSERobert Embedding Model Loaded.")
  else:
    llm_emb = None
    print("No Embedding Model Selected")
  
  return llm_emb



[중요] is_KoSimCSERobert == True 일때에 endpoint_name을 꼭 넣어 주세요

In [8]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

if is_KoSimCSERobert: endpoint_name = "<endpoint_name>"
else: endpoint_name = None

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpoint_name)

Bedrock Embeddings Model loaded


## 3. 데이터 준비
- [중요] 저자 및 동료가 아래의 웹사이트에서 크로링한 기준으로 구성 하였습니다.
- 인터넷뱅킹 FAQ > 스마트뱅킹 No.1 ~ N. 89 로 구성되었습니다.
- https://www.shinhan.com/hpe/index.jsp#050101020000

In [9]:
import pandas as pd
pd.options.display.max_rows = 20

data_file_path = "data/fsi_smart_faq_ko.csv"
df = pd.read_csv(data_file_path)
df.head()

,no,Category,Information,type,Source
0,91,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,90,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
3,89,타기관OTP 이용등록방법 알려주세요,타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다....,인터넷뱅킹,신한은행
4,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행


### 데이터 전처리

In [10]:
def process_data(df):
  ldf = df.copy()

  df_idx = ldf.drop(['no'], axis=1)
  df_idx.to_csv("data/fsi_smart_faq_ko_preprocess.csv", index=None)

  return df_idx

pre_df = process_data(df)
pre_df.head(3)

,Category,Information,type,Source
0,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행


### CSVLoader로 문서 로딩

In [11]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [12]:
loader = CSVLoader(
  file_path="data/fsi_smart_faq_ko_preprocess.csv",
  source_column="Source",
  encoding='utf-8'
)

documents_fsi = loader.load()

In [13]:
print(len(documents_fsi))
documents_fsi[0]

92


Document(page_content='Category: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 아마존은행', metadata={'source': '아마존은행', 'row': 0})

### TextSpliter로 청킹

In [14]:
if is_bedrock_embeddings:
  chunk_size = 2048
  chunk_overlap = 0
elif is_KoSimCSERobert:
  chunk_size = 800 # this is maximum
  chunk_overlap = 0

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=chunk_size,
  chunk_overlap=chunk_overlap,
  separators=["\n\n", "\n", ".", " ", ""],
  length_function=len
)

docs = text_splitter.split_documents(documents_fsi)
print(f"Number of documents after split and chunking={len(docs)}")

Number of documents after split and chunking=92


### 청킹 통계 및 내용 확인

In [15]:
from utils.rag import show_context_used, show_chunk_stat
show_chunk_stat(docs)

                 0
count    92.000000
mean    341.576087
std     175.818915
min      97.000000
25%     211.500000
50%     299.000000
75%     432.750000
max    1217.000000
Average length among 92 documents loaded is 341 characters.

Show document at maximum size
Category: 해외고객 추가인증 방법
Information: 먼저 이용에 불편을 드려 대단히 죄송합니다. 2013년 9월 전자금융사기 예방서비스 전면 시행으로 인해, 국내외에서 뱅킹이용시 거래에 따라 추가인증을 거치게 되었습니다.
① 인증서 발급/재발급, 1일 누적 300만원 이상 이체시 시행하는 추가인증 방법(누적금액은 변경될수 있습니다.)해외IP로 접속하신 부분이 자동으로 확인되면 ‘해외체류 확인’ 항목이 활성화 되기 때문에 법무부 출입국관리사무소에 출국정보를 확인 받는 부분에 동의하시면 ARS 인증없이 거래가 가능합니다.
※ 해외 소재한 국내기업의 내부 인터넷망을 이용하시는 경우 국내IP로 인식하여 해당 추가 인증기능 활성화가 안되는 경우가 있으니 유의부탁드립니다.
② 해외에서 온라인을 통한 예금해지/대출실행 업무시 추가인증 방법한국시간 기준으로 은행 영업일의 오전9시부터 오후6시 사이에 뱅킹에 접속하시면  해외 현지 연락처를 직접 입력하여 추가인증의 진행이 가능하도록 되어있습니다. 
③ 고객정보 보호 목적으로 인터넷뱅킹상에서 개인정보 변경 등의 일부 중요업무이 경우는 ARS인증만을 시행하도록 하고 있습니다.  왜냐하면, 타인에 의한 정보변경 시도 등을 차단하고자 하는 부분이므로 만약 해외에서 070 등 인터넷전화를 사용하시는 경우 미리 한국에서 정보변경을 하지 못하셨다면 고객센터 82-2-3449-8000 또는 82-2-3708-8000으로 상담사에게 문의하시기 바랍니다. 
④ “불법거래 

In [16]:
show_context_used(docs, limit=5)

-----------------------------------------------
1. Chunk: 331 Characters
-----------------------------------------------
Category: 아마존 은행의 타기관OTP 이용등록방법 알려주세요
Information: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다.
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록
- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록

 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.
type: 인터넷뱅킹
Source: 아마존은행
metadata:
 {'source': '아마존은행', 'row': 0}
-----------------------------------------------
2. Chunk: 549 Characters
-----------------------------------------------
Category: 아마존 공동인증서와 금융인증서 차이점이 무엇인가요?
Information: 공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자거래범용(수수료 4,400원) 인증서가 있으며 유효기간은 1년입니다.
아마존 공동인증서는 하드디스크나 이동식디스크, 휴대폰 등 원하시는 기기에 저장해서 이용할 수 있습니다.
인증서를 저장한 매체에서는 인증서 비밀번호로 편리하게 이용할 수 있으나 다른 기기에서 이용하려면 기기마다 복사하거나 이동식디스크에 저장해서 휴대해야 하는 불편함이 있을 수
있습니다.

아마존 금융인증서는 금융인증서는 금융결제원의 클라우드에 저장하여 이용하는 인증서로 발급/이용 시에 클라우드에 접속이 필요합니다.
금융결제원 클라우드에 연결만 가능하다면 어디서든

## 4. FAISS 벡터 Indexer 생성

In [17]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
vectorstore_faiss_fsi = FAISS.from_documents(
  documents=docs,
  embedding=llm_emb
)
print(f"vectorstore_faiss_fsi: number of elements in the index={vectorstore_faiss_fsi.index.ntotal}::")
wrapper_stroe_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_fsi)

vectorstore_faiss_fsi: number of elements in the index=92::


### 인덱싱된 벡터 확인

In [18]:
vectorstore_faiss_fsi.index_to_docstore_id
dict(list(vectorstore_faiss_fsi.docstore._dict.items())[:5])

{'63139688-966f-4a62-b288-d453371e6cb9': Document(page_content='Category: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 아마존은행', metadata={'source': '아마존은행', 'row': 0}),
 '88850a30-af02-4fe3-b8f5-b72e2faf6b7c': Document(page_content='Category: 아마존 공동인증서와 금융인증서 차이점이 무엇인가요?\nInformation: 공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자거래범용(수수료 4,400원) 인증서가 있으며 유효기간은 1년입니다. \r\n아마존 공동인증서는 하드디스크나 이동식디스크, 휴대폰 등 원하시는 기기에 저장해서 이용할 수 있습니다.\r\n인증서를 저장한 매체에서는 인증서 비밀번호로 편리하게 이용할 수 있으나 다른 기기에서 이용하려면 기기마다 복사하거나 이동식디스크에 저장해서 휴대해야 하는 불편함이 있을 수 있습니다.\r\n\r\n아마존 금융인증서는 금융인증서는 금융결제원의 클라우드에 저장하여 이용하는 인증서로 발급/이용 시에 클라우드에 접속이 필요합니다.\r\n금융결제원 클라우드에 연결만 가능하다면 어디서든 편리하게 이용 가능하지만, PC나 USB, 휴대폰 등 다른 기기로 복사는 불가합니다.(유효기간 3년/발급 수수료 무료)

## 5. 질문과 답변

In [20]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

## 6. 사용자 정의 가능한 옵션

In [23]:
from langchain.chains import RetrievalQA
from langchain import PromptTemplate

prompt_template = """
\n\nHuman: Use the following piece of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up answer

{context}

Question: {question}

\n\nAssistant:
"""

PROMPT = PromptTemplate(
  input_variables=["context", "question"], template=prompt_template
)

qa = RetrievalQA.from_chain_type(
  llm=llm_text,
  chain_type='stuff',
  retriever=vectorstore_faiss_fsi.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
  ),
  return_source_documents=True,
  chain_type_kwargs={"prompt": PROMPT}
)
query="타 기관 OTP 등록 방법 알려주세요."
result = qa({"query": query})
print_ww(result['result'])

 신한은행 인터넷뱅킹에서 타 기관 OTP를 등록하는 방법은 다음과 같습니다:

1. 인터넷뱅킹 로그인
2. 사용자관리 > 인터넷뱅킹관리 > OTP이용등록
3. 신한 쏠(SOL) 앱 로그인
4. 전체메뉴 > 설정/인증 > 이용중인 보안매체선택 > OTP이용등록

OTP 등록 후 재로그인하셔야 새로 등록된 보안매체가 적용됩니다.

궁금한 점은 신한은행 고객센터 1599-8000번으로 문의하시기 바랍니다.
